# Quick Start Guide - Azure AI Foundry

This notebook provides a hands-on introduction to Azure AI Foundry. You'll learn how to:
1. Initialize the AI Project client
2. List available models
3. Create a simple chat completion request
4. Create a basic AI agent
5. Handle basic error scenarios

## Prerequisites
- Completed environment setup from previous notebook
- Azure credentials configured

## Import Required Libraries and Setup

In the next cell, we'll:
1. Import the necessary Azure SDK libraries for authentication and AI Projects
2. Import standard Python libraries for environment variables and JSON handling
3. Initialize Azure credentials using DefaultAzureCredential
   - This will automatically use your logged-in Azure CLI credentials
   - Alternatively, it can use other authentication methods like environment variables or managed identity


In [ ]:
# Import required libraries
from azure.identity import DefaultAzureCredential, AzureCliCredential, InteractiveBrowserCredential, ChainedTokenCredential
from azure.ai.projects import AIProjectClient
import os
import json

# Initialize credentials with tenant-specific authentication (same as environment setup)
from dotenv import load_dotenv
load_dotenv()

# Get the correct tenant ID from environment
correct_tenant_id = os.getenv("TENANT_ID")
print(f"🔑 Using Tenant ID: {correct_tenant_id}")

# Create a credential chain with tenant-specific authentication
def create_credential_chain_with_tenant():
    """Create a robust credential chain for authentication with specific tenant"""
    try:
        # Try Azure CLI first with the specific tenant
        cli_credential = AzureCliCredential(tenant_id=correct_tenant_id)
        
        # Create a chained credential with fallbacks, all using the correct tenant
        credential_chain = ChainedTokenCredential(
            cli_credential,
            InteractiveBrowserCredential(tenant_id=correct_tenant_id)
        )
        
        return credential_chain
    except Exception as e:
        print(f"⚠️ Credential chain creation error: {e}")
        # Fallback to DefaultAzureCredential with tenant specified
        return DefaultAzureCredential(tenant_id=correct_tenant_id)

# Initialize credentials
try:
    credential = create_credential_chain_with_tenant()
    
    # Test the credential by getting a token for the correct tenant
    test_token = credential.get_token("https://cognitiveservices.azure.com/.default")
    print("✅ Successfully initialized Azure credentials with correct tenant!")
    
except Exception as e:
    print(f"❌ Credential initialization failed: {str(e)}")
    print(f"💡 Please run the authentication fix from the environment setup notebook first")
    credential = None

## Initialize AI Project Client

> **Note:** Before proceeding, ensure you:
> 1. Copy your `.env.example` file to `.env` from the root directory
> 2. Update the project endpoint in your `.env` file
> 3. Have a Foundry Project already provisioned in Azure AI Foundry

You can find your project endpoint in [Azure AI Foundry](https://ai.azure.com) under your project's settings:

<img src="../images/foundry-endpoint.png" alt="Project Endpoint Location" width="75%"/>



## Creating the AI Project Client

In the next cell, we'll create an AI Project client using the connection string from our `.env` file.
> **Note:** This example uses the synchronous client. For higher performance scenarios, you can also create an asynchronous client by importing `asyncio` and using the async methods from `AIProjectClient`.

The client will be used to:
- Connect to your Azure AI Project using the connection string
- Authenticate using Azure credentials
- Enable making inference requests to your deployed models


In [ ]:
from dotenv import load_dotenv
from pathlib import Path

# Load environment variables
notebook_path = Path().absolute()
parent_dir = notebook_path.parent
load_dotenv(parent_dir / '.env')

try:
    # Get the project connection string (which is actually the endpoint URL)
    project_endpoint = os.getenv("AI_FOUNDRY_PROJECT_ENDPOINT")
    if not project_endpoint:
        raise ValueError("AI_FOUNDRY_PROJECT_ENDPOINT not found in environment variables")

    print(f"🔗 Project Endpoint: {project_endpoint}")

    # Create AIProjectClient as done in environment_setup (consistent with other notebooks)
    client = AIProjectClient(
        credential=credential,
        endpoint=project_endpoint
    )
    print("✓ Successfully initialized AIProjectClient")
except Exception as e:
    print(f"× Error initializing client: {str(e)}")
    print("💡 Tip: Make sure your AI_FOUNDRY_PROJECT_ENDPOINT is set in the .env file")

## Create a Simple Completion
Let's try a basic completion request:

Now that we have an authenticated client, let's use it to make a chat completion request.
The code below demonstrates how to:
1. Get a ChatCompletionsClient from the azure-ai-inference package
2. Use it to make a simple completion request

We'll use the MODEL_DEPLOYMENT_NAME from our `.env` file, making it easy to switch between different
deployed models without changing code. This could be an Azure OpenAI model, Microsoft model, or other providers
that support chat completions.

> Note: Make sure you have the azure-ai-inference package installed (from requirements.txt or as mentioned in [README.md](../README.md#-quick-start))


In [ ]:
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

# Get environment variables - use AI Foundry project endpoint for Azure AI Inference
project_endpoint = os.getenv("AI_FOUNDRY_PROJECT_ENDPOINT")
azure_openai_key = os.getenv("AZURE_OPENAI_API_KEY")

# Use AI Foundry project endpoint with /models path for Azure AI Inference
clean_endpoint = project_endpoint.split('#')[0].strip().rstrip('/')
inference_endpoint = f"{clean_endpoint}/models"

# Create Azure AI Inference client
chat_client = ChatCompletionsClient(
    endpoint=inference_endpoint,
    credential=AzureKeyCredential(azure_openai_key)
)

# Create a simple chat completion request
response = chat_client.complete(
    model="gpt-4o",
    messages=[
        SystemMessage(content="You are a helpful health advisor."),
        UserMessage(content="How to be healthy in one sentence?")
    ]
)

print(response.choices[0].message.content)

## Create a simple Agent

Using AI Agent Service, we can create a simple agent to answer health related questions.

Let's explore Azure AI Agent Service, a powerful tool for building intelligent agents.

Azure AI Agent Service is a fully managed service that helps developers build, deploy, and scale AI agents
without managing infrastructure. It combines large language models with tools that allow agents to:
- Answer questions using RAG (Retrieval Augmented Generation)
- Perform actions through tool calling 
- Automate complex workflows

The code below demonstrates how to:
1. Create an agent with a code interpreter tool
2. Create a conversation thread
3. Send a message requesting BMI analysis 
4. Process the request and get results
5. Save any generated visualizations to local files

The agent will use the model specified in our .env file (MODEL_DEPLOYMENT_NAME) and will have access
to a code interpreter tool for creating visualizations. This showcases how agents can combine
natural language understanding with computational capabilities.

> **Note:** Generated visualizations will be saved as PNG files in the same folder as this notebook.
 



In [ ]:
from azure.ai.agents import AgentsClient
from azure.ai.agents.models import CodeInterpreterTool
from pathlib import Path

model_deployment_name = os.getenv("MODEL_DEPLOYMENT_NAME")

try:
    # Create AgentsClient
    agents_client = AgentsClient(
        endpoint=project_endpoint,
        credential=credential
    )

    # Initialize the Code Interpreter Tool
    code_interpreter = CodeInterpreterTool()

    # Create an AI agent with the code interpreter tool
    agent = agents_client.create_agent(
        model=model_deployment_name,
        name="bmi-calculator",
        instructions=(
            "You are a health analyst who calculates BMI using Python code. "
            "ALWAYS use matplotlib to create a visualization. "
            "For a BMI calculation: "
            "1. Calculate BMI = weight_lbs / (height_inches^2) * 703 "
            "2. Create a bar chart or gauge showing where the BMI falls on the scale "
            "3. Include BMI categories: Underweight (<18.5), Normal (18.5-24.9), Overweight (25-29.9), Obese (30+) "
            "4. Use matplotlib.pyplot and save the figure"
        ),
        tools=code_interpreter.definitions,
        tool_resources=code_interpreter.resources,
    )

    print(f"🟢 Created agent: {agent.id}")

    # Create a thread
    thread = agents_client.threads.create()
    print(f"🟢 Created thread: {thread.id}")

    # Create a message with explicit instruction to create visualization
    message = agents_client.messages.create(
        thread_id=thread.id,
        role="user",
        content=(
            "Calculate BMI for height 64 inches and weight 130 pounds. "
            "Create a matplotlib visualization showing: "
            "1. The calculated BMI value "
            "2. A horizontal bar or gauge showing BMI categories "
            "3. Mark where this person's BMI falls on the scale. "
            "Make sure to use plt.savefig() or display the chart."
        )
    )
    print(f"🟢 Created message: {message.id}")

    # Process the request
    run = agents_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
    print(f"🟢 Run completed: {run.status}")

    # Get the agent's response
    messages = agents_client.messages.list(thread_id=thread.id)

    # Get the initial-setup folder path (same folder as this notebook)
    download_folder = Path(__file__).parent if '__file__' in globals() else Path.cwd()
    print(f"\n📁 Download folder: {download_folder}")

    print("\n🤖 Agent Response:")
    for msg in messages:
        if msg.role == "assistant":
            # Collect text and file references
            file_ids_to_save = set()

            for content in msg.content:
                if hasattr(content, 'text'):
                    print(f"\n🤖 Assistant: {content.text.value}")
                elif hasattr(content, 'image_file'):
                    image_file_id = content.image_file.file_id
                    print(f"\n🖼️ Image file detected: {image_file_id}")
                    file_ids_to_save.add(image_file_id)

            # Some SDK responses also list attachments on message.file_ids
            if hasattr(msg, 'file_ids') and msg.file_ids:
                file_ids_to_save.update(msg.file_ids)
                print(f"\n📎 Files attached: {msg.file_ids}")

            # Save each referenced file locally
            for file_id in file_ids_to_save:
                try:
                    file_name = f"bmi_analysis_{file_id}.png"
                    agents_client.files.save(
                        file_id=file_id,
                        file_name=file_name,
                        target_dir=download_folder
                    )
                    print(f"🟢 Saved visualization: {download_folder / file_name}")
                except Exception as e:
                    print(f"⚠️ Could not save file {file_id}: {e}")
            break

    # Cleanup
    agents_client.delete_agent(agent.id)
    print(f"🟡 Cleaned up agent")

except Exception as e:
    print(f"❌ Error: {e}")